In [1]:
import pandas as pd
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
low=pd.read_csv('post_ppg_low.csv')
low

,Unnamed: 0,Trial 1:0back,Trial 6:0back
0,0,32.862,34.755
1,1,32.839,34.676
2,2,32.838,34.597
3,3,32.861,34.517
4,4,32.910,34.436
...,...,...,...
1689595,1689595,35.993,32.471
1689596,1689596,35.886,32.494
1689597,1689597,35.779,32.525
1689598,1689598,35.674,32.567


In [3]:
high=pd.read_csv('post_ppg_high.csv')

In [4]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
dfl=sc.fit_transform(low)
dfh=sc.fit_transform(high)

In [5]:
dfl=pd.DataFrame(dfl)
dfl

,0,1,2
0,-1.732050,-1.054376,-0.222184
1,-1.732048,-1.064247,-0.253381
2,-1.732046,-1.064677,-0.284579
3,-1.732044,-1.054805,-0.316172
4,-1.732042,-1.033775,-0.348160
...,...,...,...
1689595,1.732042,0.289414,-1.124158
1689596,1.732044,0.243491,-1.115075
1689597,1.732046,0.197568,-1.102833
1689598,1.732048,0.152503,-1.086246


In [6]:
dfh=pd.DataFrame(dfh)
dfh

,0,1,2
0,-1.732050,-0.041022,0.314430
1,-1.732048,-0.058876,0.297237
2,-1.732046,-0.077781,0.282432
3,-1.732044,-0.097735,0.270015
4,-1.732042,-0.118740,0.259508
...,...,...,...
1689595,1.732042,0.503532,-0.954027
1689596,1.732044,0.489879,-0.909612
1689597,1.732046,0.474650,-0.856601
1689598,1.732048,0.458372,-0.795948


In [7]:
dfl=dfl.iloc[:,1:3]
dfh=dfh.iloc[:,1:3]
dfl


,1,2
0,-1.054376,-0.222184
1,-1.064247,-0.253381
2,-1.064677,-0.284579
3,-1.054805,-0.316172
4,-1.033775,-0.348160
...,...,...
1689595,0.289414,-1.124158
1689596,0.243491,-1.115075
1689597,0.197568,-1.102833
1689598,0.152503,-1.086246


In [8]:
from scipy.stats import skew, kurtosis 

In [9]:
import numpy as np
import pandas as pd
import neurokit2 as nk
from scipy import stats
from scipy.signal import find_peaks


features = {'mean': [],'std': [],'range': [],'slope': [],'skew': [],'kurtosis': [],'heart_rate': [],'peak_amplitude': [],'num_peaks': [],'AUC': [],'IQR': [],'CV': [], 'SDNN': [],'RMSSD': []}

sampling_rate = 256  
for i in range(dfl.shape[1]):  
    for j in range(0, len(dfl), 768):  
        set_val = dfl.iloc[j:j + 768, i]
        if len(set_val) < 768:
            continue  
        signal = set_val.values
        filtered_signal = nk.signal_filter(signal, sampling_rate=sampling_rate, lowcut=0.5, highcut=10)
        time = np.arange(len(filtered_signal)) / sampling_rate
        features['mean'].append(np.mean(filtered_signal))
        features['std'].append(np.std(filtered_signal))
        peaks, _ = find_peaks(filtered_signal, height=0)  
        features['num_peaks'].append(len(peaks))
        if len(peaks) > 1:
            peak_times = time[peaks]
            hr = 60 / np.mean(np.diff(peak_times))  
            features['heart_rate'].append(hr)

           
            rr_intervals = np.diff(peak_times)
            features['SDNN'].append(np.std(rr_intervals) if len(rr_intervals) > 1 else 0)
            features['RMSSD'].append(np.sqrt(np.mean(np.square(np.diff(rr_intervals)))) if len(rr_intervals) > 1 else 0)
        else:
            features['heart_rate'].append(0) 
            features['SDNN'].append(0)
            features['RMSSD'].append(0)

        
        if peaks.size > 0:
            features['peak_amplitude'].append(np.max(filtered_signal[peaks]))
        else:
            features['peak_amplitude'].append(0)

        
        auc = np.trapz(filtered_signal, time)
        features['AUC'].append(auc)

      
        features['range'].append(np.max(filtered_signal) - np.min(filtered_signal))
        features['slope'].append(np.polyfit(time, filtered_signal, 1)[0])  
        features['skew'].append(stats.skew(filtered_signal))
        features['kurtosis'].append(stats.kurtosis(filtered_signal))
        features['IQR'].append(np.percentile(filtered_signal, 75) - np.percentile(filtered_signal, 25))
        features['CV'].append(np.std(filtered_signal) / np.mean(filtered_signal) if np.mean(filtered_signal) != 0 else 0)


features_df = pd.DataFrame(features)

In [10]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.039678,0.571302,2.687752,-0.069143,0.292720,-0.078030,177.060519,1.620853,9,0.118372,0.702053,14.398408,0.188694,0.302570
1,0.003077,0.378175,1.818487,0.001098,0.176371,0.077641,128.535565,0.990494,7,0.009566,0.433741,122.912974,0.165309,0.207752
2,-0.013320,0.633302,2.264828,0.040817,0.165533,-1.022867,123.274478,1.126555,6,-0.039416,0.991957,-47.544715,0.155380,0.233086
3,-0.013245,0.823334,3.094339,0.050719,-0.039742,-0.953162,162.909091,1.541606,8,-0.039645,1.303116,-62.161166,0.154012,0.198182
4,-0.008993,0.948394,3.381339,0.039849,-0.080793,-1.037991,90.530452,1.598439,4,-0.026329,1.499115,-105.454976,0.089380,0.117057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,-0.000567,0.758529,3.083037,0.024078,0.259523,-0.729407,148.303448,1.690678,8,-0.002206,1.106194,-1338.133338,0.170242,0.341427
4396,0.067769,0.838280,3.200806,-0.119817,0.189299,-1.024694,151.011236,1.836401,8,0.203764,1.307446,12.369668,0.197109,0.401103
4397,-0.038636,0.886364,3.284183,0.030561,0.410015,-0.985142,147.220447,1.806400,7,-0.116302,1.368470,-22.941409,0.177332,0.360477
4398,-0.051643,0.892971,3.742228,0.030199,0.371197,-0.750812,153.216958,1.911715,5,-0.154539,1.348682,-17.291361,0.141635,0.281268


In [11]:
import numpy as np
import pandas as pd
import neurokit2 as nk
from scipy import stats
from scipy.signal import find_peaks


features = {'mean': [],'std': [],'range': [],'slope': [],'skew': [],'kurtosis': [],'heart_rate': [],'peak_amplitude': [],'num_peaks': [],'AUC': [],'IQR': [],'CV': [], 'SDNN': [],'RMSSD': []}


sampling_rate = 256 

for i in range(dfh.shape[1]):  
    for j in range(0, len(dfh), 768):  
        set_val = dfh.iloc[j:j + 768, i]
        if len(set_val) < 768:
            continue  

       
        signal = set_val.values

       
        filtered_signal = nk.signal_filter(signal, sampling_rate=sampling_rate, lowcut=0.5, highcut=10)

        
        time = np.arange(len(filtered_signal)) / sampling_rate
        
       
        features['mean'].append(np.mean(filtered_signal))
        features['std'].append(np.std(filtered_signal))

       
        peaks, _ = find_peaks(filtered_signal, height=0)  
        features['num_peaks'].append(len(peaks))
        
       
        if len(peaks) > 1:
            peak_times = time[peaks]
            hr = 60 / np.mean(np.diff(peak_times))  # In beats per minute
            features['heart_rate'].append(hr)

           
            rr_intervals = np.diff(peak_times)
            features['SDNN'].append(np.std(rr_intervals) if len(rr_intervals) > 1 else 0)
            features['RMSSD'].append(np.sqrt(np.mean(np.square(np.diff(rr_intervals)))) if len(rr_intervals) > 1 else 0)
        else:
            features['heart_rate'].append(0) 
            features['SDNN'].append(0)
            features['RMSSD'].append(0)

        # Peak Amplitude
        if peaks.size > 0:
            features['peak_amplitude'].append(np.max(filtered_signal[peaks]))
        else:
            features['peak_amplitude'].append(0)

       
        auc = np.trapz(filtered_signal, time)
        features['AUC'].append(auc)

        # Range, slope, skew and kurtosis
        features['range'].append(np.max(filtered_signal) - np.min(filtered_signal))
        features['slope'].append(np.polyfit(time, filtered_signal, 1)[0])  
        features['skew'].append(stats.skew(filtered_signal))
        features['kurtosis'].append(stats.kurtosis(filtered_signal))
        features['IQR'].append(np.percentile(filtered_signal, 75) - np.percentile(filtered_signal, 25))
        features['CV'].append(np.std(filtered_signal) / np.mean(filtered_signal) if np.mean(filtered_signal) != 0 else 0)
features_dfh = pd.DataFrame(features)

In [12]:
features_dfh

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,-0.029807,0.362946,1.541858,0.041258,-0.126451,-0.624393,186.464340,0.705356,9,-0.089683,0.539213,-12.176358,0.175759,0.245996
1,-0.024297,0.262830,1.380460,0.041436,1.039285,1.328011,146.285714,0.891231,8,-0.073224,0.287956,-10.817256,0.185419,0.347960
2,-0.010412,0.359620,1.668897,0.029865,0.239242,-0.534893,142.442040,0.879954,7,-0.031288,0.553261,-34.538723,0.189623,0.236094
3,0.017474,0.303686,1.457757,-0.017448,-0.619285,0.339833,214.225941,0.593787,11,0.052322,0.344156,17.379228,0.117396,0.190037
4,-0.002948,0.319649,1.247718,0.021901,-0.270272,-0.880160,213.333333,0.611869,10,-0.008379,0.483729,-108.442467,0.059697,0.120399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.012742,0.528147,1.998929,-0.000075,0.054834,-0.950242,172.307692,1.052188,8,0.038875,0.810601,41.448114,0.127630,0.257827
4396,0.036558,0.617364,2.353480,-0.050046,0.056684,-1.022508,94.088821,1.219520,5,0.110811,0.974407,16.887068,0.223842,0.299960
4397,0.025446,0.707286,2.451825,-0.046720,0.255988,-0.995030,172.861736,1.371361,8,0.077781,1.029526,27.795780,0.140430,0.285308
4398,0.095028,0.848385,3.385876,-0.223914,0.275154,-0.850884,169.589905,2.097863,8,0.285663,1.314957,8.927732,0.145738,0.290651


In [13]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.039678,0.571302,2.687752,-0.069143,0.292720,-0.078030,177.060519,1.620853,9,0.118372,0.702053,14.398408,0.188694,0.302570
1,0.003077,0.378175,1.818487,0.001098,0.176371,0.077641,128.535565,0.990494,7,0.009566,0.433741,122.912974,0.165309,0.207752
2,-0.013320,0.633302,2.264828,0.040817,0.165533,-1.022867,123.274478,1.126555,6,-0.039416,0.991957,-47.544715,0.155380,0.233086
3,-0.013245,0.823334,3.094339,0.050719,-0.039742,-0.953162,162.909091,1.541606,8,-0.039645,1.303116,-62.161166,0.154012,0.198182
4,-0.008993,0.948394,3.381339,0.039849,-0.080793,-1.037991,90.530452,1.598439,4,-0.026329,1.499115,-105.454976,0.089380,0.117057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,-0.000567,0.758529,3.083037,0.024078,0.259523,-0.729407,148.303448,1.690678,8,-0.002206,1.106194,-1338.133338,0.170242,0.341427
4396,0.067769,0.838280,3.200806,-0.119817,0.189299,-1.024694,151.011236,1.836401,8,0.203764,1.307446,12.369668,0.197109,0.401103
4397,-0.038636,0.886364,3.284183,0.030561,0.410015,-0.985142,147.220447,1.806400,7,-0.116302,1.368470,-22.941409,0.177332,0.360477
4398,-0.051643,0.892971,3.742228,0.030199,0.371197,-0.750812,153.216958,1.911715,5,-0.154539,1.348682,-17.291361,0.141635,0.281268


In [14]:
features_dfh

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,-0.029807,0.362946,1.541858,0.041258,-0.126451,-0.624393,186.464340,0.705356,9,-0.089683,0.539213,-12.176358,0.175759,0.245996
1,-0.024297,0.262830,1.380460,0.041436,1.039285,1.328011,146.285714,0.891231,8,-0.073224,0.287956,-10.817256,0.185419,0.347960
2,-0.010412,0.359620,1.668897,0.029865,0.239242,-0.534893,142.442040,0.879954,7,-0.031288,0.553261,-34.538723,0.189623,0.236094
3,0.017474,0.303686,1.457757,-0.017448,-0.619285,0.339833,214.225941,0.593787,11,0.052322,0.344156,17.379228,0.117396,0.190037
4,-0.002948,0.319649,1.247718,0.021901,-0.270272,-0.880160,213.333333,0.611869,10,-0.008379,0.483729,-108.442467,0.059697,0.120399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.012742,0.528147,1.998929,-0.000075,0.054834,-0.950242,172.307692,1.052188,8,0.038875,0.810601,41.448114,0.127630,0.257827
4396,0.036558,0.617364,2.353480,-0.050046,0.056684,-1.022508,94.088821,1.219520,5,0.110811,0.974407,16.887068,0.223842,0.299960
4397,0.025446,0.707286,2.451825,-0.046720,0.255988,-0.995030,172.861736,1.371361,8,0.077781,1.029526,27.795780,0.140430,0.285308
4398,0.095028,0.848385,3.385876,-0.223914,0.275154,-0.850884,169.589905,2.097863,8,0.285663,1.314957,8.927732,0.145738,0.290651


In [15]:
import pywt

In [16]:

import pywt
fs=256
def extract_features(signal):
    sampling_freq = 256
    wavelet = 'db4'
    coeffs = pywt.wavedec(signal, wavelet, level=5)
    cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
    vlf_power = np.sum(np.square(cA5))
    lf_power = np.sum(np.square(cD5))
    hf_power = np.sum(np.square(cD4))
    total_power = vlf_power + lf_power + hf_power
    lf_hf_ratio = lf_power / hf_power

    return {
        'VLF': vlf_power,
        'LF': lf_power,
        'HF': hf_power,
        'total_power': total_power,
        'lf/hf':lf_hf_ratio
         }

# Process data in chunks of 768 samples
results = []
i,j,k=0,1,0
while k<2:
    while j<2201:
        while i<768*j:
            set=dfl.iloc[i:i+768,k]
            ff=extract_features(set.values)
            i=i+768
            results.append(ff)
        j=j+1
    k=k+1
    j=0
    i=0
    


In [17]:
rs=pd.DataFrame(results)

In [18]:
rs

,VLF,LF,HF,total_power,lf/hf
0,1159.670286,16.040386,2.046668,1177.757339,7.837316
1,288.926482,5.796939,0.684277,295.407698,8.471623
2,460.287920,16.431251,1.200965,477.920135,13.681711
3,1193.505752,59.341573,2.923574,1255.770899,20.297614
4,1066.387204,43.666338,2.764910,1112.818452,15.793039
...,...,...,...,...,...
4395,501.337224,23.016293,1.132012,525.485529,20.332195
4396,653.023250,23.367259,1.879115,678.269624,12.435243
4397,601.452500,38.117675,2.543910,642.114085,14.983896
4398,827.411821,39.893931,2.742582,870.048334,14.546121


In [19]:
features_df

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD
0,0.039678,0.571302,2.687752,-0.069143,0.292720,-0.078030,177.060519,1.620853,9,0.118372,0.702053,14.398408,0.188694,0.302570
1,0.003077,0.378175,1.818487,0.001098,0.176371,0.077641,128.535565,0.990494,7,0.009566,0.433741,122.912974,0.165309,0.207752
2,-0.013320,0.633302,2.264828,0.040817,0.165533,-1.022867,123.274478,1.126555,6,-0.039416,0.991957,-47.544715,0.155380,0.233086
3,-0.013245,0.823334,3.094339,0.050719,-0.039742,-0.953162,162.909091,1.541606,8,-0.039645,1.303116,-62.161166,0.154012,0.198182
4,-0.008993,0.948394,3.381339,0.039849,-0.080793,-1.037991,90.530452,1.598439,4,-0.026329,1.499115,-105.454976,0.089380,0.117057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,-0.000567,0.758529,3.083037,0.024078,0.259523,-0.729407,148.303448,1.690678,8,-0.002206,1.106194,-1338.133338,0.170242,0.341427
4396,0.067769,0.838280,3.200806,-0.119817,0.189299,-1.024694,151.011236,1.836401,8,0.203764,1.307446,12.369668,0.197109,0.401103
4397,-0.038636,0.886364,3.284183,0.030561,0.410015,-0.985142,147.220447,1.806400,7,-0.116302,1.368470,-22.941409,0.177332,0.360477
4398,-0.051643,0.892971,3.742228,0.030199,0.371197,-0.750812,153.216958,1.911715,5,-0.154539,1.348682,-17.291361,0.141635,0.281268


In [20]:
low_data=pd.concat([features_df,rs],axis=1)

In [21]:
low_data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,0.039678,0.571302,2.687752,-0.069143,0.292720,-0.078030,177.060519,1.620853,9,0.118372,0.702053,14.398408,0.188694,0.302570,1159.670286,16.040386,2.046668,1177.757339,7.837316
1,0.003077,0.378175,1.818487,0.001098,0.176371,0.077641,128.535565,0.990494,7,0.009566,0.433741,122.912974,0.165309,0.207752,288.926482,5.796939,0.684277,295.407698,8.471623
2,-0.013320,0.633302,2.264828,0.040817,0.165533,-1.022867,123.274478,1.126555,6,-0.039416,0.991957,-47.544715,0.155380,0.233086,460.287920,16.431251,1.200965,477.920135,13.681711
3,-0.013245,0.823334,3.094339,0.050719,-0.039742,-0.953162,162.909091,1.541606,8,-0.039645,1.303116,-62.161166,0.154012,0.198182,1193.505752,59.341573,2.923574,1255.770899,20.297614
4,-0.008993,0.948394,3.381339,0.039849,-0.080793,-1.037991,90.530452,1.598439,4,-0.026329,1.499115,-105.454976,0.089380,0.117057,1066.387204,43.666338,2.764910,1112.818452,15.793039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,-0.000567,0.758529,3.083037,0.024078,0.259523,-0.729407,148.303448,1.690678,8,-0.002206,1.106194,-1338.133338,0.170242,0.341427,501.337224,23.016293,1.132012,525.485529,20.332195
4396,0.067769,0.838280,3.200806,-0.119817,0.189299,-1.024694,151.011236,1.836401,8,0.203764,1.307446,12.369668,0.197109,0.401103,653.023250,23.367259,1.879115,678.269624,12.435243
4397,-0.038636,0.886364,3.284183,0.030561,0.410015,-0.985142,147.220447,1.806400,7,-0.116302,1.368470,-22.941409,0.177332,0.360477,601.452500,38.117675,2.543910,642.114085,14.983896
4398,-0.051643,0.892971,3.742228,0.030199,0.371197,-0.750812,153.216958,1.911715,5,-0.154539,1.348682,-17.291361,0.141635,0.281268,827.411821,39.893931,2.742582,870.048334,14.546121


In [25]:
low

,Unnamed: 0,Trial 1:0back,Trial 6:0back
0,0,32.862,34.755
1,1,32.839,34.676
2,2,32.838,34.597
3,3,32.861,34.517
4,4,32.910,34.436
...,...,...,...
1689595,1689595,35.993,32.471
1689596,1689596,35.886,32.494
1689597,1689597,35.779,32.525
1689598,1689598,35.674,32.567


In [27]:
fs=256
def extract_features(signal):
    sampling_freq = 256
    wavelet = 'db4'
    coeffs = pywt.wavedec(signal, wavelet, level=5)
    cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
    vlf_power = np.sum(np.square(cA5))
    lf_power = np.sum(np.square(cD5))
    hf_power = np.sum(np.square(cD4))
    total_power = vlf_power + lf_power + hf_power
    lf_hf_ratio = lf_power / hf_power

    return {
        'VLF': vlf_power,
        'LF': lf_power,
        'HF': hf_power,
        'total_power': total_power,
        'lf/hf':lf_hf_ratio
         }

# Process data in chunks of 768 samples
results = []
i,j,k=0,1,0
while k<2:
    while j<2201:
        while i<768*j:
            set=dfh.iloc[i:i+768,k]
            ff=extract_features(set.values)
            i=i+768
            results.append(ff)
        j=j+1
    k=k+1
    j=0
    i=0
    

In [32]:
rsh=pd.DataFrame(results)

In [34]:
rsh

,VLF,LF,HF,total_power,lf/hf
0,127.363313,18.626626,0.906882,146.896821,20.539188
1,395.798008,5.637300,1.002712,402.438020,5.622054
2,167.939412,10.511142,0.897428,179.347982,11.712523
3,245.202672,12.080181,0.930826,258.213678,12.977922
4,134.432231,11.350332,1.433749,147.216313,7.916539
...,...,...,...,...,...
4395,583.578899,13.611418,0.554778,597.745096,24.534871
4396,443.841205,18.755709,1.068086,463.665000,17.560115
4397,487.225026,12.286130,1.410313,500.921469,8.711633
4398,477.029560,42.822398,3.660123,523.512081,11.699717


In [36]:
high_data=pd.concat([features_dfh,rsh],axis=1)

In [38]:
high_data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,-0.029807,0.362946,1.541858,0.041258,-0.126451,-0.624393,186.464340,0.705356,9,-0.089683,0.539213,-12.176358,0.175759,0.245996,127.363313,18.626626,0.906882,146.896821,20.539188
1,-0.024297,0.262830,1.380460,0.041436,1.039285,1.328011,146.285714,0.891231,8,-0.073224,0.287956,-10.817256,0.185419,0.347960,395.798008,5.637300,1.002712,402.438020,5.622054
2,-0.010412,0.359620,1.668897,0.029865,0.239242,-0.534893,142.442040,0.879954,7,-0.031288,0.553261,-34.538723,0.189623,0.236094,167.939412,10.511142,0.897428,179.347982,11.712523
3,0.017474,0.303686,1.457757,-0.017448,-0.619285,0.339833,214.225941,0.593787,11,0.052322,0.344156,17.379228,0.117396,0.190037,245.202672,12.080181,0.930826,258.213678,12.977922
4,-0.002948,0.319649,1.247718,0.021901,-0.270272,-0.880160,213.333333,0.611869,10,-0.008379,0.483729,-108.442467,0.059697,0.120399,134.432231,11.350332,1.433749,147.216313,7.916539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.012742,0.528147,1.998929,-0.000075,0.054834,-0.950242,172.307692,1.052188,8,0.038875,0.810601,41.448114,0.127630,0.257827,583.578899,13.611418,0.554778,597.745096,24.534871
4396,0.036558,0.617364,2.353480,-0.050046,0.056684,-1.022508,94.088821,1.219520,5,0.110811,0.974407,16.887068,0.223842,0.299960,443.841205,18.755709,1.068086,463.665000,17.560115
4397,0.025446,0.707286,2.451825,-0.046720,0.255988,-0.995030,172.861736,1.371361,8,0.077781,1.029526,27.795780,0.140430,0.285308,487.225026,12.286130,1.410313,500.921469,8.711633
4398,0.095028,0.848385,3.385876,-0.223914,0.275154,-0.850884,169.589905,2.097863,8,0.285663,1.314957,8.927732,0.145738,0.290651,477.029560,42.822398,3.660123,523.512081,11.699717


In [40]:
low_data['label']=0
high_data['label']=1

In [44]:
data=pd.concat([low_data,high_data],ignore_index=True)
data

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf,label
0,0.039678,0.571302,2.687752,-0.069143,0.292720,-0.078030,177.060519,1.620853,9,0.118372,0.702053,14.398408,0.188694,0.302570,1159.670286,16.040386,2.046668,1177.757339,7.837316,0
1,0.003077,0.378175,1.818487,0.001098,0.176371,0.077641,128.535565,0.990494,7,0.009566,0.433741,122.912974,0.165309,0.207752,288.926482,5.796939,0.684277,295.407698,8.471623,0
2,-0.013320,0.633302,2.264828,0.040817,0.165533,-1.022867,123.274478,1.126555,6,-0.039416,0.991957,-47.544715,0.155380,0.233086,460.287920,16.431251,1.200965,477.920135,13.681711,0
3,-0.013245,0.823334,3.094339,0.050719,-0.039742,-0.953162,162.909091,1.541606,8,-0.039645,1.303116,-62.161166,0.154012,0.198182,1193.505752,59.341573,2.923574,1255.770899,20.297614,0
4,-0.008993,0.948394,3.381339,0.039849,-0.080793,-1.037991,90.530452,1.598439,4,-0.026329,1.499115,-105.454976,0.089380,0.117057,1066.387204,43.666338,2.764910,1112.818452,15.793039,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,0.012742,0.528147,1.998929,-0.000075,0.054834,-0.950242,172.307692,1.052188,8,0.038875,0.810601,41.448114,0.127630,0.257827,583.578899,13.611418,0.554778,597.745096,24.534871,1
8796,0.036558,0.617364,2.353480,-0.050046,0.056684,-1.022508,94.088821,1.219520,5,0.110811,0.974407,16.887068,0.223842,0.299960,443.841205,18.755709,1.068086,463.665000,17.560115,1
8797,0.025446,0.707286,2.451825,-0.046720,0.255988,-0.995030,172.861736,1.371361,8,0.077781,1.029526,27.795780,0.140430,0.285308,487.225026,12.286130,1.410313,500.921469,8.711633,1
8798,0.095028,0.848385,3.385876,-0.223914,0.275154,-0.850884,169.589905,2.097863,8,0.285663,1.314957,8.927732,0.145738,0.290651,477.029560,42.822398,3.660123,523.512081,11.699717,1


In [66]:
data.to_csv('ppg_feature.csv')

In [46]:
x=data.iloc[:,0:19]

In [48]:
x

,mean,std,range,slope,skew,kurtosis,heart_rate,peak_amplitude,num_peaks,AUC,IQR,CV,SDNN,RMSSD,VLF,LF,HF,total_power,lf/hf
0,0.039678,0.571302,2.687752,-0.069143,0.292720,-0.078030,177.060519,1.620853,9,0.118372,0.702053,14.398408,0.188694,0.302570,1159.670286,16.040386,2.046668,1177.757339,7.837316
1,0.003077,0.378175,1.818487,0.001098,0.176371,0.077641,128.535565,0.990494,7,0.009566,0.433741,122.912974,0.165309,0.207752,288.926482,5.796939,0.684277,295.407698,8.471623
2,-0.013320,0.633302,2.264828,0.040817,0.165533,-1.022867,123.274478,1.126555,6,-0.039416,0.991957,-47.544715,0.155380,0.233086,460.287920,16.431251,1.200965,477.920135,13.681711
3,-0.013245,0.823334,3.094339,0.050719,-0.039742,-0.953162,162.909091,1.541606,8,-0.039645,1.303116,-62.161166,0.154012,0.198182,1193.505752,59.341573,2.923574,1255.770899,20.297614
4,-0.008993,0.948394,3.381339,0.039849,-0.080793,-1.037991,90.530452,1.598439,4,-0.026329,1.499115,-105.454976,0.089380,0.117057,1066.387204,43.666338,2.764910,1112.818452,15.793039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,0.012742,0.528147,1.998929,-0.000075,0.054834,-0.950242,172.307692,1.052188,8,0.038875,0.810601,41.448114,0.127630,0.257827,583.578899,13.611418,0.554778,597.745096,24.534871
8796,0.036558,0.617364,2.353480,-0.050046,0.056684,-1.022508,94.088821,1.219520,5,0.110811,0.974407,16.887068,0.223842,0.299960,443.841205,18.755709,1.068086,463.665000,17.560115
8797,0.025446,0.707286,2.451825,-0.046720,0.255988,-0.995030,172.861736,1.371361,8,0.077781,1.029526,27.795780,0.140430,0.285308,487.225026,12.286130,1.410313,500.921469,8.711633
8798,0.095028,0.848385,3.385876,-0.223914,0.275154,-0.850884,169.589905,2.097863,8,0.285663,1.314957,8.927732,0.145738,0.290651,477.029560,42.822398,3.660123,523.512081,11.699717


In [50]:
y=data.iloc[:,19]

In [52]:
y

0       0
1       0
2       0
3       0
4       0
       ..
8795    1
8796    1
8797    1
8798    1
8799    1
Name: label, Length: 8800, dtype: int64

In [54]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [56]:
from sklearn.ensemble import RandomForestClassifier


In [58]:
rfc=RandomForestClassifier(n_estimators=1000)


In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 1000)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [61]:
y_pred = classifier.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [64]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred)*100)

Accuracy :  73.90151515151516
